In [2]:
import cv2
import numpy as np
import pytesseract 
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def get_string(img_path):
    # Read image with opencv
    img = cv2.imread(img_path)
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Write the image after apply opencv to do some
    cv2.imwrite('sample_ritesh_process.png', img)
    # Recognize text with tesseract for python
    img = Image.open('sample_ritesh_process.png')
    result = pytesseract.image_to_string(img, lang = 'eng')
    #print(result)

    # Writing Result into text file 
    file = open("testfile.txt","w") 
    file.write(result)
    file.close()
    return result

get_string('riteshpan.png')
#print ("------PAN READ -------")

import json
import re

filename = open('testfile.txt','r')
text = filename.read()
#`print(text)

text1 = re.sub("\s+"," ", text)
#print(text1)
#print('\n')

text2 = []
# Splitting the lines to sort the text paragraph wise
lines = text1.split(' ')
for lin in lines:
    s = lin.strip()
    s = s.rstrip()
    s = s.lstrip()
    text2.append(s)
#print(text2) 

upper = re.findall("[A-Z]+[A-Z]", text1)
#print (upper)   

# Using regex to find the neceesary information
from toolz import frequencies, valfilter

upper = re.findall("[A-Z]+[A-Z]", text1)
#print (upper)

name_filter =list(valfilter(lambda count: count > 1, frequencies(upper)).keys()) 
#print(name_filter)
FatherName = name_filter[0]
#print(FatherName)

for i in text2 :
    if i == FatherName :
        y = text2.index(i) - 1
        (text1[y])
        break
self_name = text2[y]        
#print("Self_name:",self_name)

name_filter =list(valfilter(lambda count: count > 1, frequencies(upper)).keys()) 
fathers_name = name_filter[0]
surname = name_filter[1]
#print("Fathers_name:",fathers_name)
#print("Surname:",surname)
#print("\n")

pan_name = self_name+" "+fathers_name+" "+surname
#print("Pan_name:",pan_name)

#PAN NO
id = re.compile(r'\w\w\w\w\w\d\d\d\d\w') 
ID = id.search(text1)
pan_id = ID.group()
pan_id
#print('Pan_id:',pan_id)

#DOB
dob = re.compile(r'\d\d/\d\d/\d\d\d\d') 
DOB = dob.search(text1)
pan_dob = DOB.group()
pan_dob
#print('Pan_dob:', pan_dob)
#print("\n")

# Converting the extracted informaton into json
di = {'Pan_name':pan_name,
      'Pan_id':pan_id,
      'Pan_dob':pan_dob,
      }

# Saving the json file
#print('\n',di)
with open('result.json', 'w') as fp:
    json.dump(di, fp, sort_keys=True, indent=4)
#print("\n")    

import json
import time
from pytesseract import image_to_string 
from PIL import Image
import pytesseract
from selenium import webdriver
from bs4 import BeautifulSoup as BSoup

def get_captcha_text():
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    im = Image.open('screenshot.png')
    #area = (825, 410, 1040, 490)
    area = (825, 380, 1040, 520)
    
    # Define crops points
    im = im.crop((area))
    im.save('cropped_screenshot.png')
    captcha_text = image_to_string(Image.open('cropped_screenshot.png'))
    #print('CAPTCHA:',captcha_text)
    return captcha_text


def login_to_website():
    with open('result.json') as f:
        file = json.load(f)
    pan_id = file['Pan_id'] 
    pan_name = file['Pan_name']
    pan_dob = file['Pan_dob']
    
    url = 'https://www1.incometaxindiaefiling.gov.in/e-FilingGS/Services/VerifyYourPanDeatils.html'
    driver = webdriver.Chrome(executable_path = r"C:\Program Files\chromedriver.exe")
    print('----------Opening browser----------')
    time.sleep(2)
    driver.get(url)
    driver.set_window_size(1400, 1000)
    
    while (True):
        # pan id
        user_id = driver.find_element_by_xpath('//*[@id="VerifyYourPanGSAuthentication_pan"]')
        user_id.clear()
        user_id.send_keys(pan_id)
    
        # Name of pan
        user_name = driver.find_element_by_xpath('//*[@id="VerifyYourPanGSAuthentication_fullName"]')
        user_name.clear()
        user_name.send_keys(pan_name)
    
        #dob of pan
        user_dob = driver.find_element_by_xpath('//*[@id="dateField"]')
        user_dob.clear()
        user_dob.send_keys(pan_dob)
    
        #Captcha
        driver.save_screenshot('screenshot.png')
        time.sleep(1)
        captcha = driver.find_element_by_xpath('//*[@id="VerifyYourPanGSAuthentication_captchaCode"]')
        captcha.clear()
        time.sleep(1)
        captcha_text = get_captcha_text()
        time.sleep(1)
        captcha.send_keys(captcha_text)
        time.sleep(1)
        driver.find_element_by_xpath("//body").click()
        driver.find_element_by_xpath('//*[@id="submitbtn"]').click()
        
        success = (driver.current_url == 'https://www1.incometaxindiaefiling.gov.in/e-FilingGS/Services/VerifyYourPanGSAuthentication.html')
        #print(success)
        if success == False :
            print("------Check PAN status-------")
            driver.save_screenshot('screenshot1.png')
            active_text = image_to_string(Image.open('screenshot1.png'))
            #print('Active_text:',active_text)
            #print('\n')

            if 'PANis Active and the details are matching with PAN database' in active_text:
                print("------PAN Verified-------")
                break
            elif 'Invalid Code' in active_text :
                print("---Captcha Wrong---")
                continue
            elif 'enter the code' in active_text :
                print("----Captcha box was empty please try again-----")
                driver.find_element_by_xpath('//*[@onclick="changeImg()"]').click()
                continue
            elif 'Norecord found' in active_text :
                print("---WRONG PAN.Enter correct PAN-----")
                break
                driver.quit()
            elif 'not matching' in active_text :
                print("PAN is Active but the details are not matching with PAN database")
                break
                driver.quit()
            else :
                print("something went wrong!")
                continue
        else:
            print("------Check_PAN_status-------")
            driver.save_screenshot('screenshot2.png')
            active_text2 = image_to_string(Image.open('screenshot2.png'))
            #print('Active_text2:',active_text2)
            #print('\n')

            if 'PANis Active and the details are matching with PAN database' in active_text2:
                print("------PAN Verified-------")
                break
            elif 'Invalid Code' in active_text2 :
                print("---Captcha Wrong---")
                continue
            elif 'enter the code' in active_text :
                print("----Captcha box was empty please try again-----")
                driver.find_element_by_xpath('//*[@onclick="changeImg()"]').click()
                continue
            elif 'Norecord found' in active_text2 :
                print("---WRONG PAN.Enter correct PAN-----")
                break
                driver.quit()
            elif 'not matching' in active_text2 :
                print("PAN is Active but the details are not matching with PAN database")
                break
                driver.quit()
            else :
                print("something went wrong!")
                continue

login_to_website()

----------Opening browser----------
------Check PAN status-------
----Captcha box was empty please try again-----
------Check PAN status-------
----Captcha box was empty please try again-----
------Check_PAN_status-------
---Captcha Wrong---
------Check_PAN_status-------
---Captcha Wrong---
------Check_PAN_status-------
------PAN Verified-------
